In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pysr import PySRRegressor, jl
from sklearn.model_selection import train_test_split
import sympy

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [2]:
# Ustawienie ziarna dla powtarzalności eksperymentów
np.random.seed(0)

# Dodanie pakietu Primes.jl do środowiska Julia
jl.seval("""
import Pkg
Pkg.add("Primes")
using Primes: prime
""")

# Definicja funkcji p(i) dla liczb pierwszych
jl.seval("""
function p(i::T) where T
    if i < 0
        return T(0)
    elseif i >= 1 && i < 1000
        return T(prime(round(Int, i)))
    else
        return T(1000)  # Domyślna wartość dla poza zakresem
    end
end
""")

# Klasa sympy dla funkcji p
class sympy_p(sympy.Function):
    pass

# Domyślne parametry dla PySR
default_pysr_params = dict(
   populations=30,
   model_selection="best",
)

┌ Error: curl_easy_setopt: 4
└ @ Downloads.Curl C:\Users\micha\.julia\juliaup\julia-1.11.5+0.x64.w64.mingw32\share\julia\stdlib\v1.11\Downloads\src\Curl\utils.jl:50
   Resolving package versions...
  No Changes to `C:\Users\micha\.julia\conda\3\x86_64\julia_env\Project.toml`
  No Changes to `C:\Users\micha\.julia\conda\3\x86_64\julia_env\Manifest.toml`


In [3]:
def generate_data(n_samples=200, range_min=-5, range_max=5, noise_std=0):
   """Generuje dane uczące dla funkcji f(x) = 2.2sin(x_0 + 2x_1) - x_5^2 - p(⌊x_0⌋)"""
   X = (range_max - range_min) * np.random.rand(n_samples, 6) + range_min
   
   # Obliczenie liczb pierwszych dla podłogi x_0
   p_values = np.zeros(n_samples)
   for i in range(n_samples):
       p_values[i] = float(jl.p(np.floor(X[i, 0])))
   
   y_clean = 2.2 * np.sin(X[:, 0] + 2 * X[:, 1]) - X[:, 5] ** 2 - p_values
   y = y_clean + noise_std * np.random.randn(n_samples)
   return X, y

In [4]:
def run_experiment(range_vals=(-5, 5), noise_std=0, operator_set=1):
   """
   Przeprowadza eksperyment regresji symbolicznej z podanymi parametrami.
   
   Args:
       range_vals: tupla (min, max) zakresu danych
       noise_std: odchylenie standardowe szumu
       operator_set: numer zestawu operatorów (1, 2 lub 3)
   
   Returns:
       wyniki eksperymentu w formie string
   """
   print(f"\n--- Eksperyment: zakres {range_vals}, szum {noise_std}, zestaw operatorów {operator_set} ---")
   
   # Wygeneruj dane
   X, y = generate_data(n_samples=200, range_min=range_vals[0], range_max=range_vals[1], 
                      noise_std=noise_std)
   
   # Wybór zestawu operatorów
   if operator_set == 1:
      binary_ops = ["+", "*"]
      unary_ops = ["cos", "exp", "sin", "p"]
      maxsize = 20
      constraints = None
   elif operator_set == 2:
      binary_ops = ["+", "*", "-", "^"]
      unary_ops = ["cos", "exp", "sin", "log", "p"]
      maxsize = 30
      constraints = {"^": [(-100.0, 100.0), (-100.0, 100.0)]}
   else:  # operator_set == 3
      binary_ops = ["+", "*", "-", "^"]
      unary_ops = ["exp", "sin", "p"]
      maxsize = 15
      constraints = {"^": [(-100.0, 100.0), (-100.0, 100.0)]}
   
   # Konfiguracja i trening modelu
   model = PySRRegressor(
      niterations=30,
      binary_operators=binary_ops,
      unary_operators=unary_ops,
      maxsize=maxsize,
      constraints=constraints,
      extra_sympy_mappings={"p": sympy_p},
      **default_pysr_params,
   )
   
   model.fit(X, y)
   
   # Wypisz wyniki
   print_string = ""
   print_string += f"\n=== Eksperyment: zakres {range_vals}, szum {noise_std}, zestaw operatorów {operator_set} ==="
   print_string += "\nTrzy najlepsze rozwiązania (według score):"
   best_by_score = model.equations_.sort_values('score', ascending=False).head(3)
   iterator = 0
   for i, row in best_by_score.iterrows():
      iterator += 1
      print_string += f"\n{iterator}. {row['equation']} (score: {row['score']:.4f}, loss: {row['loss']:.6f})"
   
   print_string += "\nNajlepsze rozwiązanie (best):"
   print_string += f"\n1. {model.sympy()}"
   
   return print_string


In [5]:
def run_all_experiments():
   results = []
   
   # Eksperymenty na 3.0 (podstawowe)
   print("\n### EKSPERYMENTY NA 3.0 (BEZ SZUMU) ###")
   for op_set in [1, 2, 3]:
      results.append(run_experiment(range_vals=(-5, 5), noise_std=0, operator_set=op_set))
   
   print("\n### EKSPERYMENTY NA 3.0 (Z SZUMEM 0.5) ###")
   for op_set in [1, 2, 3]:
      results.append(run_experiment(range_vals=(-5, 5), noise_std=0.5, operator_set=op_set))
   
   # Eksperymenty na 4.0 (szerszy zakres i większy szum)
   print("\n### EKSPERYMENTY NA 4.0 (SZUM 2.0) ###")
   for op_set in [1, 2, 3]:
      results.append(run_experiment(range_vals=(-15, 15), noise_std=2, operator_set=op_set))
   
   print("\n### EKSPERYMENTY NA 4.0 (SZUM 5.0) ###")
   for op_set in [1, 2, 3]:
      results.append(run_experiment(range_vals=(-15, 15), noise_std=5, operator_set=op_set))
   
   return results

In [6]:
results = run_all_experiments()


### EKSPERYMENTY NA 3.0 (BEZ SZUMU) ###

--- Eksperyment: zakres (-5, 5), szum 0, zestaw operatorów 1 ---


c:\Users\micha\.julia\conda\3\x86_64\Lib\site-packages\pysr\sr.py:2776: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
Compiling Julia backend...
[ Info: Started!



Expressions evaluated per second: 7.630e+04
Progress: 419 / 900 total iterations (46.556%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           6.977e+04  1.594e+01  y = -84.869
3           6.968e+04  6.760e-04  y = x₃ + -84.757
4           1.048e+02  6.499e+00  y = p(x₀) * -1.0123
6           5.523e+01  3.203e-01  y = (p(x₀) * -1.0047) + -7.3384
8           5.493e+01  2.752e-03  y = ((p(x₀) * -1.005) + x₂) + -7.4697
9           5.125e+01  6.932e-02  y = ((p(x₀) * -1.0045) + cos(x₅)) + -7.1851
11          5.125e+01  8.941e-08  y = ((cos(x₅) + -6.9099) + (p(x₀) * -1.0045)) + -0.27523
12          4.827e+01  6.001e-02  y = cos(x₅) + (-7.3384 + ((p(x₀) * -1.0047) + cos(x₅)))
13          4.591e+01  4.997e-02  y = cos(x₅ * 0.66802) + (((p(x₀) + -8.6973) * -1.0043) + -...
                    

[ Info: Final population:
[ Info: Results saved to:


───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           6.977e+04  1.594e+01  y = -84.869
3           6.968e+04  6.760e-04  y = x₃ + -84.757
4           1.048e+02  6.499e+00  y = p(x₀) * -1.0123
6           5.523e+01  3.203e-01  y = (p(x₀) + 7.3038) * -1.0047
8           3.599e+00  1.365e+00  y = (p(x₀) + (x₅ * x₅)) * -0.99965
10          3.412e+00  2.681e-02  y = ((p(x₀) + (x₅ * x₅)) * -1.0002) + 0.45582
11          3.242e+00  5.110e-02  y = (((x₅ * x₅) + sin(x₀)) + p(x₀)) * -0.99989
13          3.083e+00  2.506e-02  y = -0.99989 * (((x₅ * x₅) + sin(x₀)) + (p(x₀) + -0.60795)...
                                      )
14          2.824e+00  8.769e-02  y = (p(x₀) * -1.0006) + (-0.97274 * ((x₅ * x₅) + (x₀ * -0....
                                      28067)))
16          2.707e+00  2.129e-02  y = ((p(x₀) * -1.0006) + (x₀ * 0.27301)) + ((x₅ * (x₅ * -0...
                                    

...


Expressions evaluated per second: 8.630e+04
Progress: 411 / 900 total iterations (45.667%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           6.515e+04  1.594e+01  y = -146.07
3           6.468e+04  3.663e-03  y = x₁ + -145.92
4           4.265e+03  2.719e+00  y = -72.156 - p(x₀)
6           4.108e+03  1.870e-02  y = (x₁ + -72.015) - p(x₀)
8           3.275e+01  2.416e+00  y = (0.36061 - p(x₀)) - (x₅ * x₅)
9           3.261e+01  4.281e-03  y = (cos(x₁) - p(x₀)) - (x₅ * x₅)
10          3.101e+01  5.035e-02  y = (0.2471 - p(x₀ * 0.97559)) - (x₅ * x₅)
11          3.094e+01  2.266e-03  y = (p(-1.2263) - p(0.97559 * x₀)) - (x₅ * x₅)
12          3.090e+01  1.383e-03  y = ((0.31065 - p(x₀ * 0.97863)) * 1.0022) - (x₅ * x₅)
13          3.002e+01  2.879e-02  y = (sin(x₀ * 0.35172) - p(x₀ * 0.9

[ Info: Final population:
[ Info: Results saved to:
c:\Users\micha\.julia\conda\3\x86_64\Lib\site-packages\pysr\sr.py:2776: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


  - outputs\20250512_002149_G7HHqc\hall_of_fame.csv

Expressions evaluated per second: 1.050e+05
Progress: 586 / 900 total iterations (65.111%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           4.764e+04  1.594e+01  y = -132.03
3           4.737e+04  2.879e-03  y = -130.6 - x₃
4           4.541e+03  2.345e+00  y = -78.338 - p(x₀)
6           4.426e+03  1.282e-02  y = (x₁ - p(x₀)) - 78.292
8           3.021e+01  2.493e+00  y = (0.6135 - p(x₀)) - (x₅ * x₅)
10          3.011e+01  1.726e-03  y = (0.58965 - p(x₀)) - (x₅ * (x₅ - -0.036359))
12          2.976e+01  5.809e-03  y = (-0.83667 - p(x₀ - -0.038975)) - ((x₅ * x₅) + -1.4813)
───────────────────────────────────────────────────────────────────────────────────────────────────
════════════════════════════════════════════════════════════

[ Info: Started!
[ Info: Final population:
[ Info: Results saved to:


───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           4.764e+04  1.594e+01  y = -132.03
3           4.737e+04  2.879e-03  y = -130.62 - x₃
4           4.541e+03  2.345e+00  y = -78.338 - p(x₀)
6           4.426e+03  1.282e-02  y = (x₁ - p(x₀)) - 78.292
8           3.021e+01  2.493e+00  y = (0.61306 - p(x₀)) - (x₅ * x₅)
9           3.021e+01  1.137e-04  y = (sin(x₄) - p(x₀)) - (x₅ * x₅)
10          2.977e+01  1.486e-02  y = (0.72173 - p(x₀ - -0.038975)) - (x₅ * x₅)
11          2.976e+01  1.137e-04  y = (sin(0.72295) - p(x₀ - -0.038975)) - (x₅ * x₅)
12          2.936e+01  1.363e-02  y = (0.5443 - p(x₀)) - ((x₅ * x₅) + sin(p(x₄)))
15          2.921e+01  1.655e-03  y = (0.72173 - p(x₀ + 0.037403)) - ((x₅ * x₅) - sin(x₁ + -...
                                      1.1503))
───────────────────────────────────────────────────────────────────────────────────────────────────
  - outputs\20250512

In [7]:
print("\n\n### PODSUMOWANIE WSZYSTKICH EKSPERYMENTÓW ###")
for result in results:
   print(result)
   print("-" * 80)



### PODSUMOWANIE WSZYSTKICH EKSPERYMENTÓW ###

=== Eksperyment: zakres (-5, 5), szum 0, zestaw operatorów 1 ===
Trzy najlepsze rozwiązania (według score):
1. p(x0) * -1.0122799 (score: 6.4995, loss: 104.812355)
2. (p(x0) + (x5 * x5)) * -0.9996523 (score: 1.3654, loss: 3.599497)
3. (p(x0) + 7.303805) * -1.0047319 (score: 0.3203, loss: 55.234028)
Najlepsze rozwiązanie (best):
1. (x5*x5 + sympy_p(x0))*(-0.9996523)
--------------------------------------------------------------------------------

=== Eksperyment: zakres (-5, 5), szum 0, zestaw operatorów 2 ===
Trzy najlepsze rozwiązania (według score):
1. -8.170675 - p(x0) (score: 7.1995, loss: 63.663666)
2. (0.4298719 - (x5 * x5)) - p(x0) (score: 2.8412, loss: 3.658588)
3. ((0.4777713 - (x5 * x5)) - cos((x1 + x1) - ((x0 - -1.5376787) * -1.0096959))) - p(x0) (score: 0.2006, loss: 1.953838)
Najlepsze rozwiązanie (best):
1. -x5*x5 - sympy_p(x0) - cos(x1 + x1 - (-1.0096959)*(x0 - 1*(-1.5376787))) + 0.4777713
---------------------------------